#### Adds in stellar radii for systems whose stellar radii are missing in the exoarchive catalog but not in Gaia data release 2. Also, the code classifies any system whose exoplanets were detected by more than one method as 'Mixed'.

In [1]:
import csv
import numpy as np

from astropy import constants as apc

MJUPITER = apc.M_jup.value
MEARTH = apc.M_earth.value
RJUPITER = apc.R_jup.value
REARTH = apc.R_earth.value

In [2]:
class System:
    def __init__(self, sysid=None):
        self.sysid = sysid

In [3]:
#reading in file
my_file = 'C:/Users/djhoo/Documents/Oviraptor-master/catalogues/oviraptor_crossmatch_catalog_v2.csv'  
#my_file = "/Users/research/projects/oviraptor/catalogues/oviraptor_crossmatch_catalog.csv"

raw_data = []
with open(my_file) as infile:
    reader = csv.reader(infile)
    
    for row in reader:
        raw_data.append(row)
        
KEYS = raw_data[0]
VALUES = raw_data[1:]
new_values = []
for i in range(len(VALUES)):
    if i % 2 == 1:
        new_values.append(VALUES[i])
VALUES = new_values

def get_key(keyname, keys = KEYS, values = VALUES):
    i = keys.index(keyname)
    arr = []
    for row in values:
        arr.append(row[i])
    return np.array(arr)

d = {}
d['starname'] = get_key('hostname')

d['rp'] = get_key('pl_rade')
d['rp_err1'] = get_key('pl_radeerr1')
d['rp_err2'] = get_key('pl_radeerr2')
d['mp'] = get_key('pl_bmasse')
d['mp_err1'] = get_key('pl_bmasseerr1')
d['mp_err2'] = get_key('pl_bmasseerr2')

d['ror'] = get_key('pl_ratror')
d['ror_err1'] = get_key('pl_ratrorerr1')
d['ror_err2'] = get_key('pl_ratrorerr2')

d['rs'] = get_key('st_rad')
d['rs_err1'] = get_key('st_raderr1')
d['rs_err2'] = get_key('st_raderr2')
d['ms'] = get_key('st_mass')
d['ms_err1'] = get_key('st_masserr1')
d['ms_err2'] = get_key('st_masserr2')

d['p'] = get_key('pl_orbper')
d['disc_method'] = get_key('discoverymethod')
d['temp'] = get_key('st_teff')
d['mag'] = get_key('sy_vmag')
d['ref'] = get_key('st_refname')
d['instrument'] = get_key('disc_facility')

print('Number of planets in crossmatched catalog: %d' %len(d['starname']))

Number of planets in crossmatched catalog: 4107


# Classify systems as mixed

In [4]:
def check_uniformity(detection_methods):
    default_disc = detection_methods[0]
    status = True
    for i in range(len(detection_methods)):
        if detection_methods[i] != default_disc:
            status = False
    return status

d['system_disc_method'] = []

for i in range(len(d['starname'])):
    if check_uniformity(d['disc_method'][d['starname']==d['starname'][i]]) == False:
        d['system_disc_method'].append('Mixed')
    else:
        d['system_disc_method'].append(d['disc_method'][i])

In [5]:
WRITENEW = True
if WRITENEW:
    filepath = "C:/Users/djhoo/Documents/Oviraptor-master/catalogues/oviraptor_master.csv"

    with open(filepath, "w") as outfile:
        writer = csv.writer(outfile)
        writer.writerow(d.keys())
        writer.writerows(zip(*d.values()))
        
print('Writing complete!')

Writing complete!
